In [2]:
from random import random, sample
import numpy as np

from astropy import coordinates
from astropy.io import fits
from astroquery.mast import Observations
from astropy.table import vstack, Table

import matplotlib.pyplot as plt
from PIL import Image

import sys
import os
import time
import re
import json
 
import requests
from urllib.parse import quote as urlencode

In [3]:
#Görüntüleri 512*512'lik karelere bölen ve 
#her bir kesiti 0-1 aralığına normalize eden fonksiyon
def process_image(given_image: np.ndarray, SIZE: int):

    img = given_image
    
    rows = img.shape[0]
    cols = img.shape[1]

    
    #Çok küçük görseller elenir, kullanılmaz.
    if (rows < 512) or (cols <512):
        print(rows)
        return -1

    
    #Tüm nan değerler 0'a eşitlenir.
    img[np.isnan(img)] = 0

    
    #Görsel boyutlarının 512'ye tam bölünmesi sağlanır.
    del_rows = (rows // SIZE) * SIZE
    del_cols = (cols // SIZE) * SIZE

    img = img[:del_rows, :del_cols]
    rows = img.shape[0]
    cols = img.shape[1]

    
    #Görsel 512*512'lik kesitlere bölünür.
    sections = (img
              .reshape(rows // SIZE, SIZE, -1, SIZE)
              .swapaxes(1, 2)
              .reshape(-1, SIZE, SIZE))

    
    #Her bir kesit 0-1 aralığına normalize edilir ve sonuç döndürülür.
    res = []
    for i in range(sections.shape[0]):
        new = sections[i]
        new = new - np.min(new)
        max = np.max(new)
        new = new/max
        res.append(new)
        
    return np.array(res)